In [49]:
import numpy as np
import pandas as pd
import json
from pathlib import Path

data=json.loads(Path("./queries.txt").read_text())


In [ ]:
print(data)

In [52]:
features=np.array(data["features"], dtype=int)

outputs=np.array(data["outputs"], dtype=int).reshape(-1,1)

train_features=features[:]
train_outputs=outputs[:]
# train_features=np.hstack((train_features, np.zeros((train_features.shape[0], 1)) ))
for i in range(7):
    train_features=np.vstack((train_features, np.zeros((1, train_features.shape[1]))))
    train_features=np.vstack((train_features, np.ones((1, train_features.shape[1]))))
    train_outputs=np.vstack((train_outputs,[[i],[i]]))

train_features=train_features.astype(bool).astype(int)

print(train_features.shape)
print(train_outputs.shape)
test_features=features[:]
test_features=test_features.astype(bool).astype(int)
test_outputs=outputs[:]

m=train_features.shape[0]
f=train_features.shape[1]

(305, 671)
(305, 1)


In [40]:
#naive bayes algorithm

pys=np.zeros((7))

for out in train_outputs:
    pys[out[0]]+=1


pfeatures=np.zeros((7,train_features.shape[1]))

for ind, feature in enumerate(train_features):

    pfeatures[train_outputs[ind]]+=feature

pfeatures/=pys.reshape(-1,1)
pys/=m
logpys=np.log(pys)
logpfeatures=np.log(pfeatures)


In [67]:
word_mapper=data["word-mapper"]
value_mapper=data["value-mapper"]

map_value={}

for key,value in value_mapper.items():
    map_value[value]=key

In [117]:
#testing our algorithm

def process(query):
    query=query.lower()
    new_query=[]
    for word in query.split():
        new_query.append(word if word[-1].isalpha() else word[:-1])
    new_query=" ".join(new_query)
    return new_query
    
query="an official demanded bribe in exchange of official documents"
query=process(query)
print(query)

query_feature=np.zeros((1,train_features.shape[1]))

for word in query.split():
    query_feature[0][word_mapper.get(word, f-1)]+=1

query_feature=np.tile(query_feature,(7,1))

predict=pfeatures**query_feature*((1-pfeatures)**(1-query_feature))
predict=np.log(predict).sum(axis=1).reshape(-1,1)+logpys.reshape(-1,1)

# print(predict)
prediction=np.argmax(predict)
print(map_value[prediction])




an official demanded bribe in exchange of official documents
Anti-Bribery
